In [1]:
from data_transformers import chain
import pandas as pd
from analysislib import (
    get_geonomenclador, 
    exportar_definitivo,
    exportar_transformador,
    match_relocate,
)

from data_transformers.default_transformers import (
    drop_col,
    wide_to_long,
    replace_value,
    rename_cols,
    sort_values,
    cast_col
)

geonomenclador = get_geonomenclador()

def compare_ids(a):
    def _(b):
        try:
            a_before, a_after = a.split('_g')
            b_before, b_after = b.split('_g')
            return a_before == b_before and int(a_after) == int(b_after)
        except Exception:
            return False
    return _

In [2]:
work = pd.read_excel('./work.xlsx', sheet_name='Suma de gráficos')
tipo = work[['ID', 'Unnamed: 7', 'Bajada','Publicación']]

In [3]:
SUBTOP = 'CRECIM'
entrega = 1
folder = f'../output/{SUBTOP}{entrega}'
grafico_n = 2

archivo = f'{SUBTOP}_g{grafico_n:02d}'
df = pd.read_csv(f'{folder}/definitivos/{archivo}.csv')

exportar = (lambda df:
                exportar_definitivo(archivo=archivo,
                                    folder=folder,
                                    df=df,
                                    nuevo_nombre=None))

_, tipologia_target, va, bajada = tipo[tipo.ID.map(compare_ids(archivo))].iloc[0]

va = va.lower() != 'no va'

print(f"Va? = {'Sí' if va else 'No'}")
print(f"Tipología target = '{tipologia_target}'")
print(f"Bajada = '{bajada}'")

mapping = pd.read_csv(f'{folder}/internal_mapping.csv')
mapping = {id:archivo for (_,_,archivo,id,_) in mapping.iloc}

plantilla = pd.read_excel(f'../tmp/ArgenData-{SUBTOP}.xlsx', header=6)
plantilla_slice = plantilla[plantilla.dataset_archivo == mapping[archivo]]

primary_keys = plantilla_slice.variable_nombre[plantilla_slice.primary_key].unique().tolist()
print('Primary keys:', ', '.join(f'"{x}"' for x in primary_keys))
df.head()

Va? = Sí
Tipología target = 'Scatter con geoselector y colores'
Primary keys: "iso3", "anio"


,iso3,pais_nombre,continente_fundar,anio,expectativa_al_nacer,pib_pc,nivel_agregacion
0,ABW,Aruba,"América del Norte, Central y el Caribe",1990,73.08,30823.48,pais
1,ABW,Aruba,"América del Norte, Central y el Caribe",1991,73.10,32222.66,pais
2,ABW,Aruba,"América del Norte, Central y el Caribe",1992,73.18,32986.54,pais
3,ABW,Aruba,"América del Norte, Central y el Caribe",1993,73.23,34336.55,pais
4,ABW,Aruba,"América del Norte, Central y el Caribe",1994,73.27,35984.83,pais


In [10]:
pipeline = chain(
    drop_col('pais_nombre'),
    drop_col('continente_fundar'),
    drop_col('nivel_agregacion'),
    wide_to_long(primary_keys),
    rename_cols({'iso3': 'geocodigo'})
)

callstack, result = pipeline(df)

result.head()

,geocodigo,anio,indicador,valor
0,ABW,1990,expectativa_al_nacer,73.08
1,ABW,1991,expectativa_al_nacer,73.10
2,ABW,1992,expectativa_al_nacer,73.18
3,ABW,1993,expectativa_al_nacer,73.23
4,ABW,1994,expectativa_al_nacer,73.27


In [11]:
exportar_transformador(archivo, pipeline, callstack)

'CRECIM_g02_transformer.py'